<a href="https://colab.research.google.com/github/ranjith13119/CNN/blob/main/Tomato_Leaf_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"
# %cd /content/drive/MyDrive/CNN/Tomato-disease-Classficaion1
# !kaggle datasets download -d noulam/tomato

In [ ]:
# !unzip \*.zip  && rm *.zipy

In [ ]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = "/content/drive/MyDrive/CNN/Tomato-disease-Classficaion1/New Plant Diseases Dataset(Augmented)/train"
val_path = "/content/drive/MyDrive/CNN/Tomato-disease-Classficaion1/New Plant Diseases Dataset(Augmented)/valid"
folders = glob(train_path+"/*")
working_path = "/content/drive/MyDrive/CNN/Tomato-disease-Classficaion1/"
strModelFileName = "models/InceptionV3.h5"

In [ ]:
def model_inceptionV3():
  inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  for layers in inception.layers:
    layers.trainable = False
  x = Flatten()(inception.output)
  prediction = Dense(len(folders), activation='softmax')(x)
  model = Model(inputs=inception.input, outputs=prediction)
  return(model)

In [ ]:
def model_RestNet50():
  resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  for layers in resnet.layers:
    layers.trainable = False
  x = Flatten()(resnet.output)
  prediction = Dense(len(folders), activation='softmax')(x)
  model = Model(inputs=resnet.input, outputs=prediction)
  return(model)

In [ ]:
import os
# Make sure that previous "best network" is deleted.
def deleteSavedNet(best_weights_filepath):
	if(os.path.isfile(best_weights_filepath)):
		os.remove(best_weights_filepath)
		print("deleteSavedNet():File removed")
	else:
		print("deleteSavedNet():No file to remove") 

In [ ]:
#inception_model = model_inceptionV3()



deleteSavedNet(working_path + strModelFileName)
if not os.path.exists(working_path + "models"):
  os.makedirs(working_path + "models")
    
if not os.path.exists(working_path + strModelFileName):
  inception_model = model_inceptionV3()
  # model = model_RestNet50()
else:
  model = load_model(working_path + strModelFileName)

deleteSavedNet():No file to remove
87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
inception_model.summary()

In [ ]:
inception_model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
                    rotation_range=40, 
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest'
                )

In [ ]:
validation_datagen = ImageDataGenerator(
                        rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        fill_mode='nearest'
                      )

In [ ]:
train_generator= train_datagen.flow_from_directory(train_path, target_size = (224, 224),batch_size = 32, class_mode = 'categorical')

Found 18345 images belonging to 10 classes.


In [ ]:
val_generator= validation_datagen.flow_from_directory(val_path, target_size = (224, 224),batch_size = 32, class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [ ]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TerminateOnNaN, LambdaCallback
checkpoint = ModelCheckpoint(working_path + strModelFileName, monitor='val_acc', verbose=1, save_best_only=True,  mode='auto', save_weights_only=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.0001)
call_savemodel_BA = LambdaCallback(on_epoch_end= lambda epochs, logs: model.save_weights(working_path + strModelFileName) if logs.get('acc')>0.99 else None)
call_terminate_NAN = TerminateOnNaN()
earlystopping_callback = EarlyStopping(monitor='val_acc',verbose=1, min_delta=0.5, patience=3, baseline=None)
callbacks_list = [ checkpoint, reduce_lr, call_savemodel_BA, call_terminate_NAN, earlystopping_callback ]

In [ ]:
model_final = inception_model.fit_generator(
  train_generator,
  validation_data=val_generator,
  epochs=10,
  steps_per_epoch = STEP_SIZE_TRAIN,
  validation_steps = STEP_SIZE_VALID,
  callbacks=callbacks_list)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model_final.history['loss'], label='train loss')
plt.plot(model_final.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(model_final.history['accuracy'], label='train acc')
plt.plot(model_final.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
def model_predict(img_path, model):
    print(img_path)
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)

    preds = model.predict(x)
    preds=np.argmax(preds, axis=1)
    if preds==0:
        preds="The Disease is Tomato___Bacterial_spot"
    elif preds==1:
        preds="The Disease is Tomato___Early_blight"
    elif preds==2:
        preds="The Disease is Tomato___Late_blight"
    elif preds==3:
        preds="Te Disease is Tomato___Leaf_Mold"
    elif preds==4:
        preds="The Disease is Tomato___Septoria_leaf_spot"
    elif preds==5:
        preds="The Disease is Tomato___Spider_mites Two-spotted_spider_mite"
    elif preds==6:
        preds="The Disease is Tomato___Target_Spot"
    elif preds==7:
        preds="The Disease is Tomato___Tomato_Yellow_Leaf_Curl_Virus"
    elif preds==8:
        preds="The Disease is Tomato___Tomato_mosaic_virus"
    elif preds==9:
        preds="The Disease is Tomato___healthy"
    
    return preds

In [ ]:
y_pred = model_final.predict(test_set)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
path = "/content/drive/MyDrive/CNN/Tomato-disease-Classficaion1/New Plant Diseases Dataset(Augmented)/valid/Tomato___Target_Spot/01e0b8b1-e713-4c6d-973b-f7636280c58a___Com.G_TgS_FL 9816_newPixel25.JPG"
output = model_predict(path, model_final)
print(output)